# 🔎 Investigation Assistant: Company & Person Scraper

This Python-based investigation assistant automates the collection and analysis of publicly available information about companies or individuals from online sources, such as the UK Companies House.

**Key Features:**
- **Automated Web Scraping:** Collects structured and unstructured data from web pages.
- **AI-powered Analysis:** Uses Groq (LLM) to extract relevant fields like company status, registered office, contacts, affiliations, SIC codes, and incorporation dates.
- **Prompt-engineered Extraction:** Optimized prompts ensure accurate, structured JSON outputs with minimal missing data.
- **Excel Reporting:** Generates well-formatted Excel files for records, review, and further analysis.
- **Interactive Exploration:** Investigate multiple companies or individuals in a single session.

*Designed for investigative research, compliance checks, and corporate intelligence, combining traditional web scraping with AI-assisted data extraction.*


# 🔐 Securely enter Groq API key

In [45]:
import os
from getpass import getpass
GROQ_API_KEY = getpass("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
print("Groq API key set successfully (hidden).")


Enter your Groq API key: ··········
Groq API key set successfully (hidden).


# Install required Libraries

In [35]:
!pip install langchain groq requests beautifulsoup4 lxml pandas tqdm -q

In [44]:
import os
import re
import json
import requests
import datetime
from bs4 import BeautifulSoup
from getpass import getpass
import pandas as pd
from IPython.display import display, Markdown
from groq import Groq


#GROQ_API_KEY = getpass("Enter your Groq API key: ")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# -----------------------------
# Search Companies House for company profile links
# -----------------------------
def search_companies_house(company_name):
    search_url = f"https://find-and-update.company-information.service.gov.uk/search?q={company_name.replace(' ', '+')}"
    r = requests.get(search_url, headers={"User-Agent": "Mozilla/5.0"}, timeout=15)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "lxml")

    links = []
    for a in soup.find_all("a", href=True):
        if re.match(r"^/company/\d+", a["href"]):
            full_url = "https://find-and-update.company-information.service.gov.uk" + a["href"]
            if full_url not in links:
                links.append(full_url)
    return links

# -----------------------------
# Scrape text snippets from a URL
# -----------------------------
def scrape_url(url):
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=15)
        r.raise_for_status()
    except Exception as e:
        return [f"⚠️ Error fetching {url}: {str(e)}"]

    soup = BeautifulSoup(r.text, "html.parser")
    texts = []

    # Add headers, paragraphs, list items
    for tag in soup.find_all(["h1","h2","h3","p","li","dt","dd","span"]):
        text = tag.get_text(strip=True)
        if text and len(text.split()) > 1:
            texts.append(text)

    return texts[:100]  # increase snippets to capture more details


# -----------------------------
# Filter snippets by exact company name
# -----------------------------
def filter_snippets_by_target(snippets, target_name):
    pattern = re.compile(re.escape(target_name), re.IGNORECASE)
    filtered = [s for s in snippets if pattern.search(s)]
    return filtered if filtered else snippets

# -----------------------------
# Analyze with Groq (Updated)
# -----------------------------
def analyze_snippets_with_groq(target, url, snippets):
    system_prompt = f"""
You are an expert investigator AI. You are given snippets from a UK Companies House company profile page.
Your task is to extract exact information for the target company '{target}' and return a JSON object ONLY with these fields:

- name_detected: Official company name
- registered_office_address: Full registered address
- company_status: e.g., Active, In Administration, Dissolved
- company_type: e.g., Private limited company
- incorporation_date: Date of incorporation
- accounts_next: Next accounts date
- accounts_last: Last accounts date
- confirmation_statement_next: Next confirmation statement date
- confirmation_statement_last: Last confirmation statement date
- nature_of_business_sic: SIC code(s) and description

**Important Instructions:**
- Only extract data that is explicitly present in the snippets.
- If data is missing, return an empty string.
- Return valid JSON only, no extra text.
- Use correct date formats if available (YYYY-MM-DD or 'DD Month YYYY').
"""

    joined_snippets = "\n".join(snippets)
    user_msg = f"Target: {target}\nURL: {url}\nSnippets:\n{joined_snippets}"

    resp = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user", "content": user_msg},
        ],
        temperature=0.2,
        max_tokens=600,
    )

    return resp.choices[0].message.content.strip()

# -----------------------------
# Step 7: Full investigation pipeline
# -----------------------------
def investigate_company(target):
    print(f"\n🔍 Searching Companies House for '{target}'...")
    urls = search_companies_house(target)
    if not urls:
        print("❌ No company profiles found.")
        return None

    results = {
        "target": target,
        "queried_at": datetime.datetime.now(datetime.timezone.utc).isoformat(),
        "items": []
    }

    for url in urls:
        print(f"\n📂 Processing: {url}")
        snippets = scrape_url(url)
        snippets = filter_snippets_by_target(snippets, target)
        analysis = analyze_snippets_with_groq(target, url, snippets)
        results["items"].append({
            "url": url,
            "snippets": snippets,
            "analysis": analysis
        })
    return results

# -----------------------------
#Beautified Pretty-print results (Updated)
# -----------------------------
def pretty_print_results(res):
    display(Markdown(f"## 🔹 Investigation Results for: **{res['target']}**"))
    display(Markdown(f"Queried at: {res['queried_at']}\n"))

    for i, item in enumerate(res["items"], start=1):
        try:
            analysis_json = json.loads(re.search(r"\{.*\}", item['analysis'], re.DOTALL).group())
        except:
            analysis_json = {
                "name_detected":"",
                "registered_office_address":"", "company_status":"", "company_type":"",
                "incorporation_date":"", "accounts_next":"", "accounts_last":"",
                "confirmation_statement_next":"", "confirmation_statement_last":"",
                "nature_of_business_sic":""
            }

        def highlight_missing(val):
            return f"**❌ Missing**" if not val else val

        md_text = f"""
<details>
<summary>📂 Result #{i} - {highlight_missing(analysis_json.get('name_detected',''))} (Click to expand)</summary>

**URL:** {item['url']}

- ✅ **Name Detected:** {highlight_missing(analysis_json.get('name_detected',''))}
- 🏢 **Registered Office Address:** {highlight_missing(analysis_json.get('registered_office_address',''))}
- 📊 **Company Status:** {highlight_missing(analysis_json.get('company_status',''))}
- 🏷️ **Company Type:** {highlight_missing(analysis_json.get('company_type',''))}
- 📅 **Incorporation Date:** {highlight_missing(analysis_json.get('incorporation_date',''))}
- 📈 **Accounts (Next):** {highlight_missing(analysis_json.get('accounts_next',''))}
- 📉 **Accounts (Last):** {highlight_missing(analysis_json.get('accounts_last',''))}
- 📅 **Confirmation Statement (Next):** {highlight_missing(analysis_json.get('confirmation_statement_next',''))}
- 📅 **Confirmation Statement (Last):** {highlight_missing(analysis_json.get('confirmation_statement_last',''))}
- ⚙️ **Nature of Business (SIC):** {highlight_missing(analysis_json.get('nature_of_business_sic',''))}

</details>
"""
        display(Markdown(md_text))

# -----------------------------
# Interactive loop + Excel export (Updated)
# -----------------------------
while True:
    company_name = input("\nEnter company or person name (or type 'exit' to quit): ").strip()
    if company_name.lower() == "exit":
        print("\n👋 Exiting investigation assistant. Goodbye!")
        break

    res = investigate_company(company_name)
    if res:
        pretty_print_results(res)

        # Save results to Excel with all new fields
        rows = []
        for item in res["items"]:
            try:
                analysis_json = json.loads(re.search(r"\{.*\}", item['analysis'], re.DOTALL).group())
            except:
                analysis_json = {}
            rows.append({
                "Name Detected": analysis_json.get("name_detected",""),
                "Registered Office Address": analysis_json.get("registered_office_address",""),
                "Company Status": analysis_json.get("company_status",""),
                "Company Type": analysis_json.get("company_type",""),
                "Incorporation Date": analysis_json.get("incorporation_date",""),
                "Accounts Next": analysis_json.get("accounts_next",""),
                "Accounts Last": analysis_json.get("accounts_last",""),
                "Confirmation Statement Next": analysis_json.get("confirmation_statement_next",""),
                "Confirmation Statement Last": analysis_json.get("confirmation_statement_last",""),
                "Nature of Business SIC": analysis_json.get("nature_of_business_sic",""),
                "URL": item["url"]
            })
        df = pd.DataFrame(rows)
        filename = f"investigation_{company_name.replace(' ','_')}.xlsx"
        df.to_excel(filename, index=False)
        print(f"\n📁 Saved beautifully formatted Excel: {filename}")

    # Ask if user wants to continue
    continue_choice = input("\nDo you want to investigate another company/person? (yes/no): ").strip().lower()
    if continue_choice not in ["yes", "y"]:
        print("\n👋 Exiting investigation assistant. Goodbye!")
        break



Enter company or person name (or type 'exit' to quit): 79th Group

🔍 Searching Companies House for '79th Group'...

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/11725522

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/10196600

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/15864010

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/13649781

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/13770396

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/14628949

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/14495240

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/09146736

📂 Processing: https://find-and-update.company-information.service.gov.uk/company/12795490

📂 Processing: https://find-and-update.company-information.servic

## 🔹 Investigation Results for: **79th Group**

Queried at: 2025-09-18T08:58:53.049616+00:00



<details>
<summary>📂 Result #1 - GRUDGE MATCH MANAGEMENT LTD (Click to expand)</summary>

**URL:** https://find-and-update.company-information.service.gov.uk/company/11725522

- ✅ **Name Detected:** GRUDGE MATCH MANAGEMENT LTD
- 🏢 **Registered Office Address:** Flat 1 Elsworth Court, 3 Cottonham Close, London, England, N12 9HN
- 📊 **Company Status:** Company status
- 🏷️ **Company Type:** Private limited Company
- 📅 **Incorporation Date:** 13 December 2018
- 📈 **Accounts (Next):** 5 April 2025 due by 5 January 2026
- 📉 **Accounts (Last):** 5 April 2024
- 📅 **Confirmation Statement (Next):** 22 March 2026 due by 5 April 2026
- 📅 **Confirmation Statement (Last):** 22 March 2025
- ⚙️ **Nature of Business (SIC):** 70229 - Management consultancy activities other than financial management

</details>



<details>
<summary>📂 Result #2 - 79TH GROUP CLIENT LTD (Click to expand)</summary>

**URL:** https://find-and-update.company-information.service.gov.uk/company/10196600

- ✅ **Name Detected:** 79TH GROUP CLIENT LTD
- 🏢 **Registered Office Address:** **❌ Missing**
- 📊 **Company Status:** **❌ Missing**
- 🏷️ **Company Type:** **❌ Missing**
- 📅 **Incorporation Date:** **❌ Missing**
- 📈 **Accounts (Next):** **❌ Missing**
- 📉 **Accounts (Last):** **❌ Missing**
- 📅 **Confirmation Statement (Next):** **❌ Missing**
- 📅 **Confirmation Statement (Last):** **❌ Missing**
- ⚙️ **Nature of Business (SIC):** **❌ Missing**

</details>


...


<details>
<summary>📂 Result #17 - 79TH LUXURY LIVING LIMITED (Click to expand)</summary>

**URL:** https://find-and-update.company-information.service.gov.uk/company/10787951

- ✅ **Name Detected:** 79TH LUXURY LIVING LIMITED
- 🏢 **Registered Office Address:** The Lexicon 10-12 Mount Street, Manchester, M2 5NT
- 📊 **Company Status:** In Administration
- 🏷️ **Company Type:** Private limited Company
- 📅 **Incorporation Date:** 24 May 2017
- 📈 **Accounts (Next):** 30 December 2024 due by 30 September 2025
- 📉 **Accounts (Last):** 31 December 2023
- 📅 **Confirmation Statement (Next):** 22 February 2026 due by 8 March 2026
- 📅 **Confirmation Statement (Last):** 22 February 2025
- ⚙️ **Nature of Business (SIC):** 68100 - Buying and selling of own real estate

</details>



<details>
<summary>📂 Result #18 - 79TH LUXURY LIVING (BIRKENHEAD) LTD (Click to expand)</summary>

**URL:** https://find-and-update.company-information.service.gov.uk/company/12654970

- ✅ **Name Detected:** 79TH LUXURY LIVING (BIRKENHEAD) LTD
- 🏢 **Registered Office Address:** Southport Business Park, Wight Moss Way, Southport, England, PR8 4HQ
- 📊 **Company Status:** Active
- 🏷️ **Company Type:** Private limited Company
- 📅 **Incorporation Date:** 9 June 2020
- 📈 **Accounts (Next):** 31 December 2024 due by 30 September 2025
- 📉 **Accounts (Last):** 31 December 2023
- 📅 **Confirmation Statement (Next):** 25 August 2025 due by 8 September 2025
- 📅 **Confirmation Statement (Last):** 25 August 2024
- ⚙️ **Nature of Business (SIC):** 68100 - Buying and selling of own real estate

</details>



<details>
<summary>📂 Result #19 - 79TH LUXURY LIVING EIGHT LTD (Click to expand)</summary>

**URL:** https://find-and-update.company-information.service.gov.uk/company/16252882

- ✅ **Name Detected:** 79TH LUXURY LIVING EIGHT LTD
- 🏢 **Registered Office Address:** Southport Business Park, Wight Moss Way, Southport, England, PR8 4HQ
- 📊 **Company Status:** **❌ Missing**
- 🏷️ **Company Type:** Private limited Company
- 📅 **Incorporation Date:** 14 February 2025
- 📈 **Accounts (Next):** 14 November 2026
- 📉 **Accounts (Last):** **❌ Missing**
- 📅 **Confirmation Statement (Next):** 27 February 2026
- 📅 **Confirmation Statement (Last):** **❌ Missing**
- ⚙️ **Nature of Business (SIC):** 96090 - Other service activities not elsewhere classified

</details>



<details>
<summary>📂 Result #20 - 79TH LUXURY LIVING FIVE LTD (Click to expand)</summary>

**URL:** https://find-and-update.company-information.service.gov.uk/company/14254854

- ✅ **Name Detected:** 79TH LUXURY LIVING FIVE LTD
- 🏢 **Registered Office Address:** C/O Kroll Advisory Ltd The Shard, 32 London Bridge Street, London, SE1 9SG
- 📊 **Company Status:** In Administration
- 🏷️ **Company Type:** Private limited Company
- 📅 **Incorporation Date:** 25 July 2022
- 📈 **Accounts (Next):** 31 December 2024 due by 30 September 2025
- 📉 **Accounts (Last):** 31 December 2023
- 📅 **Confirmation Statement (Next):** 24 July 2025 due by 7 August 2025
- 📅 **Confirmation Statement (Last):** 24 July 2024
- ⚙️ **Nature of Business (SIC):** 96090 - Other service activities not elsewhere classified

</details>



📁 Saved beautifully formatted Excel: investigation_79th_Group.xlsx

Do you want to investigate another company/person? (yes/no): no

👋 Exiting investigation assistant. Goodbye!


## 🚀 Next Phase

- **Import URLs** from Excel or CSV files.  
- **Improve model accuracy** by training it to extract details from multiple websites.  
- **Analyze extracted data** to enable informed decision-making.  
- **Forward processed output** to another agent for further actions.
